In [1]:
#1st block
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.5.0-bin-hadoop3'
!pip install -q findspark
!pip install pyngrok

Mounted at /content/drive


In [ ]:
!pip install streamlit

In [2]:
#2nd Block
!ngrok authtoken 2ZDM0s9kUZbJtua2DayJyPs0zCv_5zQvVrspkVxhFqWUdPqkm
!ngrok config add-authtoken 2ZDM0s9kUZbJtua2DayJyPs0zCv_5zQvVrspkVxhFqWUdPqkm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [84]:
#4th Block
!streamlit run streamlit_app.py &>/dev/null&

In [85]:
#5th Block
!ngrok http 8501 &

In [17]:
#6th block
!ps -eaf | grep streamlit

root        4125       1 27 13:08 ?        00:00:01 /usr/bin/python3 /usr/local/bin/streamlit run st
root        4156     214  0 13:09 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root        4158    4156  0 13:09 ?        00:00:00 grep streamlit


In [18]:
#7th Block
!kill -9 4125

In [83]:
#3rd Block

%%writefile streamlit_app.py
import streamlit as st
#init pyspark
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path
import pathlib
from pyspark.ml.feature import VectorAssembler
from xgboost.spark import SparkXGBRegressor,SparkXGBClassifier
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier,OneVsRest
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import DecisionTreeClassificationModel, RandomForestClassificationModel, LogisticRegressionModel
from pyspark.ml.classification import GBTClassificationModel,OneVsRestModel

spark = SparkSession.builder.master("local[*]").appName("Colab").config('spark.ui.port', '4050').getOrCreate()

st.title("Lifepulse Application (Human Activity Recognition)")

selected_model=st.selectbox("Which model you want to use?",("Decision Tree Classifier","Random Forest","GBT", "Logistic Regression","XGBoost"))
if selected_model:
  st.write("We will use the {} model to give the Prediction. ".format(selected_model))

uploaded_file=st.file_uploader("Choose a file containing the sensor data: ")
if uploaded_file:
  df_pandas=pd.read_csv(uploaded_file)
  df = spark.createDataFrame(df_pandas)
  window_spec = Window.orderBy("timestamp")
  window_size = 10
  lagged_cols = [F.lag(col, i).over(window_spec).alias(f"{col}_lag{i}") for i in range(window_size,0,-1) for col in ["timestamp","back_x", "back_y", "back_z", "thigh_x", "thigh_y", "thigh_z", "label",]]
  lagged_df = df.select(*lagged_cols).na.drop()

  filtered_df = lagged_df.filter(
        (lagged_df.label_lag1 == lagged_df.label_lag2) &
        (lagged_df.label_lag2 == lagged_df.label_lag3) &
        (lagged_df.label_lag3 == lagged_df.label_lag4) &
        (lagged_df.label_lag4 == lagged_df.label_lag5) &
        (lagged_df.label_lag5 == lagged_df.label_lag6) &
        (lagged_df.label_lag6 == lagged_df.label_lag7) &
        (lagged_df.label_lag7 == lagged_df.label_lag8) &
        (lagged_df.label_lag8 == lagged_df.label_lag9) &
        (lagged_df.label_lag9 == lagged_df.label_lag10)
  )
  vector_assembler = VectorAssembler(
        inputCols=[f"{col}_lag{i}" for i in range(window_size, 0, -1) for col in ["back_x", "back_y", "back_z", "thigh_x", "thigh_y", "thigh_z"]],
        outputCol="features"
  )
  lagged_df2 = vector_assembler.transform(filtered_df)
  final_df = lagged_df2.select("features", "label_lag1").withColumnRenamed("label_lag1", "label")
  final_df.show(truncate=False)
  (training,testing) = final_df.randomSplit([0.3,0.7])
  if selected_model=="Decision Tree Classifier":
    new_dt = DecisionTreeClassificationModel.load('/content/dt/dt')
  elif selected_model=="Random Forest":
    new_dt = RandomForestClassificationModel.load('/content/rf/dt/')
  elif selected_model=="GBT":
    new_dt = OneVsRestModel.load('/content/gbt/gbt/')
  elif selected_model=="Logistic Regression":
    new_dt = LogisticRegressionModel.load('/content/lr/lr/')
  elif selected_model=="XGBoost":
    new_dt = OneVsRestModel.load('/content/xgb/xgb/')
  else:
    pass
  predictions2 = new_dt.transform(testing)
  activity_dict={1:'walking', 3: 'shuffling', 4: 'stairs_ascending',5: 'stairs_descending', 6: 'standing', 7: 'sitting',8: 'lying'}
  p=predictions2.select("prediction", "label").take(5)
  pred=[]
  actual=[]
  for i in range(0,len(p)):
    pred.append(activity_dict[p[i][0]])
    actual.append(activity_dict[p[i][1]])
  df_pred = pd.DataFrame(list(zip(pred, actual)), columns=["Prediction", "Actual"])
  st.write(df_pred)
  evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
  accuracy = evaluator.evaluate(predictions2)
  #st.write("Test Error = %g " % (1.0 - accuracy))
  st.write("Accuracy = %g " % accuracy)
  evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
  f1 = evaluator.evaluate(predictions2)
  st.write("F1-Score = %g " % f1)


Overwriting streamlit_app.py


In [ ]:
!unzip /content/xgb.zip -d /content/xgb

In [65]:
!rm -rf xgb.zip